# Exercise 04: Bayesian Inverse Problems

In this notebook we finally get started with Bayesian inverse problems. In particular, we see how to define the postrior distribution and how to sample it using CUQIpy.

## Learning objectives
* Define posterior distrubition in CUQIpy.
* Sample posterior distribution with specific sampler and analyze results.
* Compute point estimates of posterior, e.g., ML, MAP.
* See how the high-level BayesianProblem essentially wraps the above points for convinience.

## Table of contents
1. [Defining the posterior distribution](#posterior)
2. [Sampling the posterior](#sampling)
3. [Computing point estimates of posterior](#pointestimates)
4. [Connection to BayesianProblem](#BayesianProbem)


## 1. Defining the posterior distribution <a class="anchor" id="posterior"></a>

As before, we first import the packages we need

In [ ]:
import numpy as np
import cuqi

%load_ext autoreload
%autoreload 2

### Model and data
For this example let us revisit the Deconvolution testproblem and extract a CUQIpy model and some data. Similar to earlier, we can also get additional information from the 3rd output argument (probInfo).

In [ ]:
n = 50
model, data, probInfo = cuqi.testproblem.Deconvolution.get_components(dim=n)
probInfo

### Likelihood and prior
From the problem info string above, we see that the noise is additive Gaussian with std 0.05.
Hence, as we saw in exercise 03 the likelihood distribution can be defined as the distribution of the noise with the model as mean.

In [ ]:
likelihood = cuqi.distribution.Gaussian(mean=model,std=0.05)

In this case, let us assume the ground truth is well-represented by a Gaussian distribution.
Hence we can define a Gaussian CUQIpy distribution as the prior.

In [ ]:
prior = cuqi.distribution.Gaussian(mean=np.zeros(n),std=0.1)

### Combine into posterior
Once we have the likelihood, prior and an observed set of data we have all components to define the posterior distribution. This is then simply done as follows.

In [ ]:
posterior = cuqi.distribution.Posterior(likelihood,prior,data)

#### Try yourself (optional):  
The posterior is essentially just another cuqi distribution. Have a look at `help(posterior)` to see what attributes and methods are available. What happends if you call the `sample` method?

In [ ]:
# Your code here




## 2. Posterior sampling <a class="anchor" id="sampling"></a>
In CUQIpy we provide a number of samplers in the sampler module. All samplers have the same signature, that is
`Sampler(target,..)`, where `target` is the target cuqi distribution.

For example, we can use the Linear randomize-then-optimize sampler to sample the posterior by first setting up the sampler

In [ ]:
sampler = cuqi.sampler.Linear_RTO(posterior)

and then running the sampler and storing the samples in the variable `samples`.

In [ ]:
samples = sampler.sample(500)

The CUQIpy samples object has, as we have seen a number of methods available. In this case we are interested to evaluate if the sampling went well. To do this we can have a look at the chain for 2 different values.

In [ ]:
samples.plot_chain([30,45]);

In both cases the chains look very good without very much burn-in. This is due to the how the linear_RTO sampler works. For the sake of presentation let us remove the first 100 samples using the `burnthin` method and store the burnthinned samples in a new variable

In [ ]:
samples_burnthinned = samples.burnthin(100)

Finally we can plot a confidence interval of the samples and compare to the exact solution

In [ ]:
samples_burnthinned.plot_ci(95,exact=probInfo.exactSolution)

### Trying out other samples

The Linear_RTO sampler is very good at sampling Gaussian posteriors. However, it is possible to try other samplers.

Other samplers to try:
* NUTS - A well established sampler. Can be slow. Requires gradients (do we have that?)
* pCN - works OK with enough samples (>5000)
* CWMH - works OK with enough samples (>5000)

#### Try yourself (optional):  
Try sampling the same posterior as above using the NUTS, CWMH or pCN sampler

In [ ]:
# Your code here




## 3 Computing point estimates

In [ ]:
help(posterior.loglikelihood_function)

In [ ]:
solver_ML = cuqi.solver.maximize(posterior.loglikelihood_function,np.zeros(n))
x_ML, info = solver_ML.solve()

In [ ]:
x_ML = cuqi.samples.CUQIarray(x_ML,geometry=cuqi.geometry.Continuous1D(n))
x_ML.plot()

## MAP estimation

In [ ]:
MAP = cuqi.solver.maximize(posterior.logpdf,np.zeros(n))
x_map,info = MAP.solve()
x_map = cuqi.samples.CUQIarray(x_map,geometry=cuqi.geometry.Continuous1D(n))
x_map.plot()

## High-level interface

In [ ]:
BP = cuqi.problem.BayesianProblem(likelihood,prior,data)

In [ ]:
samples = BP.sample_posterior(5000)

In [ ]:
samples.plot_ci(95,exact=probInfo.exactSolution)

In [ ]:
x_sol = BP.ML()[0]
x_sol = cuqi.samples.CUQIarray(x_sol,geometry=cuqi.geometry.Continuous1D(n))
x_sol.plot()

In [ ]:
x_map = BP.MAP()
x_map.plot()

## Using geometries to parametrize x and improve pCN sampling result.

In [ ]:
#TODO
# ADD DOC TO POSTERIOR